In [ ]:
import json

# Example with a non-ASCII character in the data
data = {'name': 'John Doe', 'city': '北京'}

# Convert Python object to JSON string
json_string = json.dumps(data)

# Encode JSON string to bytes using UTF-8
json_bytes = json_string.encode('utf-8')

# Now you can safely use `json_bytes` for writing to file or transmitting over network
print(json_bytes)

In [1]:
import uuid

In [3]:
str(uuid.uuid4())

'18512de5-26b5-43b2-b9fc-0b4dc959a33d'

In [16]:
spark.stop()

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName('sudhirs')
    .config('spark.jars.packages', "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0")
    .getOrCreate()
)

24/07/05 20:30:44 WARN Utils: Your hostname, iPhone.local resolves to a loopback address: 127.0.0.1; using 192.168.1.4 instead (on interface en0)
24/07/05 20:30:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/homebrew/Cellar/apache-spark/3.5.1/libexec/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/crepantherx/.ivy2/cache
The jars for the packages stored in: /Users/crepantherx/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0f9212b3-2177-4ffd-8260-4d0c759eb320;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.0 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in cen

In [3]:
df = spark.readStream \
.format('kafka') \
.option('kafka.bootstrap.servers', 'localhost:9092') \
.option('subscribe', 'users_created') \
.option('startingOffsets', 'earliest') \
.load()

In [7]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
schema = StructType([
    StructField("id", StringType(), False),
    StructField("first_name", StringType(), False),
    StructField("last_name", StringType(), False),
    StructField("gender", StringType(), False),
    StructField("address", StringType(), False),
    StructField("post_code", StringType(), False),
    StructField("email", StringType(), False),
    StructField("username", StringType(), False),
    StructField("registered_date", StringType(), False),
    StructField("phone", StringType(), False),
    StructField("picture", StringType(), False)
])

In [10]:
df.start().selectExpr("CAST(value AS STRING)").select(from_json(col('value'), schema).alias('data')).select("data.*").show()

AttributeError: 'DataFrame' object has no attribute 'start'

In [15]:
df.selectExpr("CAST(value AS STRING)") \
  .select(from_json(col('value'), schema).alias('data')) \
  .select("data.*").writeStream.format("console").start()

24/07/05 20:41:16 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/0d/q901nqg95plg2cz08tmt75480000gn/T/temporary-a72ef63d-2b60-4c53-bc3f-ef919853dba4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/07/05 20:41:16 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
